Starter code to try the Mediapipe
face mesh model: adds a complete mapping of the face. The model outputs an estimate of 478 3-dimensional face landmarks

In [ ]:
import cv2
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

# for the camera capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break


    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(frame_rgb)

# draw landmarks if any face is detected
    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(
                frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

    # video stream
    cv2.imshow('Mimotion Face Mesh', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
import cv2
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()
mp_drawing = mp.solutions.drawing_utils

# كاميرا
cap = cv2.VideoCapture(0)


LEFT_EYE = [33, 133]
RIGHT_EYE = [362, 263]
LEFT_EYEBROW = [55, 65]
RIGHT_EYEBROW = [285, 295]
MOUTH = [61, 291]

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break


    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)


    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape
            # عين يسار
            for idx in LEFT_EYE:
                x, y = int(landmarks.landmark[idx].x * w), int(landmarks.landmark[idx].y * h)
                cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)
            # عين يمين
            for idx in RIGHT_EYE:
                x, y = int(landmarks.landmark[idx].x * w), int(landmarks.landmark[idx].y * h)
                cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)
            # الحاجب يسار
            for idx in LEFT_EYEBROW:
                x, y = int(landmarks.landmark[idx].x * w), int(landmarks.landmark[idx].y * h)
                cv2.circle(frame, (x, y), 3, (255, 0, 0), -1)
            # الحاجب يمين
            for idx in RIGHT_EYEBROW:
                x, y = int(landmarks.landmark[idx].x * w), int(landmarks.landmark[idx].y * h)
                cv2.circle(frame, (x, y), 3, (255, 0, 0), -1)
            # الفم
            for idx in MOUTH:
                x, y = int(landmarks.landmark[idx].x * w), int(landmarks.landmark[idx].y * h)
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)

    cv2.imshow("Selected Points", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# start with MediaPipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

cap = cv2.VideoCapture(0)

# the points of face landmarks
LEFT_EYE = [33, 133]
RIGHT_EYE = [362, 263]
MOUTH = [61, 291]
LEFT_EYEBROW = [55, 65]
RIGHT_EYEBROW = [285, 295]

def calc_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def predict_emotion(left_eye_open, right_eye_open, mouth_open, left_eyebrow_lift, right_eyebrow_lift):
    if mouth_open > 25 and left_eye_open > 10 and right_eye_open > 10:
        return "Surprised 😲"
    elif mouth_open > 20 and left_eye_open > 5 and right_eye_open > 5:
        return "Happy 😁"
    elif left_eyebrow_lift < 10 and right_eyebrow_lift < 10:
        return "Sad 😢"
    elif left_eyebrow_lift > 20 and right_eyebrow_lift > 20:
        return "Angry 😠"
    else:
        return "Neutral 😐"

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape

            def get_landmark(idx):
                return (int(face_landmarks.landmark[idx].x * w), int(face_landmarks.landmark[idx].y * h))

            left_eye = [get_landmark(LEFT_EYE[0]), get_landmark(LEFT_EYE[1])]
            right_eye = [get_landmark(RIGHT_EYE[0]), get_landmark(RIGHT_EYE[1])]
            mouth = [get_landmark(MOUTH[0]), get_landmark(MOUTH[1])]
            left_eyebrow = [get_landmark(LEFT_EYEBROW[0]), get_landmark(LEFT_EYEBROW[1])]
            right_eyebrow = [get_landmark(RIGHT_EYEBROW[0]), get_landmark(RIGHT_EYEBROW[1])]

            # calculate distances
            left_eye_open = calc_distance(left_eye[0], left_eye[1])
            right_eye_open = calc_distance(right_eye[0], right_eye[1])
            mouth_open = calc_distance(mouth[0], mouth[1])
            left_eyebrow_lift = calc_distance(left_eyebrow[0], left_eyebrow[1])
            right_eyebrow_lift = calc_distance(right_eyebrow[0], right_eyebrow[1])

            
            emotion = predict_emotion(left_eye_open, right_eye_open, mouth_open, left_eyebrow_lift, right_eyebrow_lift)

            
            cv2.putText(frame, f'Emotion: {emotion}', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 255), 2)

    cv2.imshow('Mimotion - Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [2]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)


cap = cv2.VideoCapture(0)


LEFT_EYE_INDICES = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_INDICES = [362, 385, 387, 263, 373, 380]
MOUTH_INDICES = [78, 308, 13, 14]

def calc_eye_aspect_ratio(eye_points):
    # حساب النسبة بين الطول والعرض للعين
    p1, p2, p3, p4, p5, p6 = eye_points
    vertical1 = np.linalg.norm(np.array(p2) - np.array(p6))
    vertical2 = np.linalg.norm(np.array(p3) - np.array(p5))
    horizontal = np.linalg.norm(np.array(p1) - np.array(p4))
    ear = (vertical1 + vertical2) / (2.0 * horizontal)
    return ear

def calc_mouth_open(mouth_points):
    # حساب فتح الفم
    top = np.array(mouth_points[2])
    bottom = np.array(mouth_points[3])
    distance = np.linalg.norm(top - bottom)
    return distance

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape

            def get_point(idx):
                return (int(landmarks.landmark[idx].x * w), int(landmarks.landmark[idx].y * h))

            left_eye = [get_point(i) for i in LEFT_EYE_INDICES]
            right_eye = [get_point(i) for i in RIGHT_EYE_INDICES]
            mouth = [get_point(i) for i in MOUTH_INDICES]

            left_ear = calc_eye_aspect_ratio(left_eye)
            right_ear = calc_eye_aspect_ratio(right_eye)
            mouth_open = calc_mouth_open(mouth)

            avg_ear = (left_ear + right_ear) / 2.0

            # تحديد المشاعر 
            emotion = "Neutral 😐"
            if mouth_open > 30 and avg_ear > 0.25:
                emotion = "Surprised 😲"
            elif mouth_open > 20:
                emotion = "Happy 😁"
            elif avg_ear < 0.2:
                emotion = "Sad 😢"
            else:
                emotion = "Neutral 😐"

            cv2.putText(frame, f'Emotion: {emotion}', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

    cv2.imshow('Improved Mimotion - Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 